# Creating a new dataset
In this notebook we will have a look at creating a new dataset from a set of records. First we will take a look a the result of a training, make a selection based on the results and than train on the new dataset.

In [ ]:
# Imports
import panel as pn

from icevision.all import *
import icedata
from icevision_dashboards.dashboards import *
from icevision_dashboards.data import *

In [ ]:
pn.extension()

# Look at training results


In [ ]:
trainings_results = ObjectDetectionResultsDataset.load("test_data/fridge_example")

If we take a look at the data below we can see that the model works better on images with only one annoation. Furthermore we can see, that the score distribution that the model is much more confident in some classes. (Here the reason is that the training was stopped before it could converge). Lets say we now want to create a subset and train a model on the subset, where the number of annotations equals 1 and we want to use the following classes Pug, Shiba Inu, Great Pyrenees ans Sphynx. We can use the `ObjectDetectionDatasetGenerator` dashboard to do this.

In [ ]:
trainings_resuls_overivew = ObjectDetectionResultOverview(trainings_results)
trainings_resuls_overivew.show_loss_tab()

# Create a subdataset

In [ ]:
# load the data and class_map
path = icedata.fridge.load_data()
class_map = icedata.fridge.class_map()

# use the provided dataset parser
parser = icedata.fridge.parser(data_dir=path)
records = parser.parse(RandomSplitter([1]))[0]

In [ ]:
# create a dataset the can be consumed by a dashboard
dash_ds = BboxRecordDataset(records, class_map=class_map)

First select the ranges with the range silder than select the classes you want.

In [ ]:
dash_generator = ObjectDetectionDatasetGeneratorRange(dash_ds, width=1000, height=700)
dash_generator.show()

In [ ]:
# load the created dataset
new_dataset = BboxRecordDataset("datasets/dataset.json")

In [ ]:
# quick overview of the dataset
ObjectDetectionDatasetOverview(new_dataset, height=700, width=1000).show()

In [ ]:
# split the records into a training and test dataset
train_records, valid_records = new_dataset.split_in_train_and_val(0.8)

## Train a model on the new Dataset
Now that we have the new dataset we can train a model on the data.

In [ ]:
# define transforms
train_tfms = tfms.A.Adapter(
    [*tfms.A.aug_tfms(size=384, presize=512), tfms.A.Normalize()]
)
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(384), tfms.A.Normalize()])

In [ ]:
# create the datasets and dataloaders
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

train_dl = faster_rcnn.train_dl(train_ds, batch_size=8, num_workers=4, shuffle=True)
valid_dl = faster_rcnn.valid_dl(valid_ds, batch_size=8, num_workers=4, shuffle=False)

In [ ]:
# create a model and train it for some epochs
model = faster_rcnn.model(num_classes=len(new_dataset.class_map))

learn = faster_rcnn.fastai.learner(dls=[train_dl, valid_dl], model=model)

learn.fine_tune(5, base_lr=1e-4)

In [ ]:
samples_plus_losses, preds, losses_stats = faster_rcnn.interp.plot_top_losses(model=model, dataset=valid_ds, sort_by="loss_total", n_samples=1)

In [ ]:
# create a dataset that can be consumed by the analysis dashboard
valid_result_ds = ObjectDetectionResultsDataset.init_from_preds_and_samples(preds, samples_plus_losses, class_map=class_map)

In [ ]:
# create the dashboard
result_overview_dash = ObjectDetectionResultOverview(valid_result_ds)

In [ ]:
# show the loss tab
result_overview_dash.show_loss_tab()

In [ ]:
# show the ap tab
result_overview_dash.show_ap_tab()